# CV and Cover Letter Automation System

This notebook provides a step-by-step interface for tailoring your CV and cover letter based on job descriptions using OpenAI's API.

## How it works:

1. set up your OpenAI API key
2. Paste a job description
3. The system will analyze your existing CV and cover letter
4. OpenAI will suggest tailored content based on the job description
5. New cv documents will be created in the output folder

Let's get started!

## 1. Setup and Configuration

First, let's import the necessary libraries and set up our environment.

In [1]:
import os
import sys
import json
from datetime import datetime
from IPython.display import display, HTML

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import our custom modules
from src.parsers.document_parser import CVParser, CoverLetterParser
from src.utils.openai_integration import OpenAIIntegration
from src.updaters.document_updater import DocumentUpdater

# Define paths
DATA_DIR = os.path.join(project_root, 'data')
OUTPUT_DIR = os.path.join(project_root, 'output')

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Setup complete!")

Setup complete!


## 2. Set up OpenAI API Key

You'll need an OpenAI API key to use this system. If you don't have one, you can get it from [OpenAI's website](https://platform.openai.com/account/api-keys).

You can either set it as an environment variable or enter it directly below:

In [2]:
# Option 1: Set API key directly
import getpass
api_key = getpass.getpass("Enter your OpenAI API key (input will be hidden): ")

# Initialize OpenAI integration
openai_integration = OpenAIIntegration(api_key=api_key)

# Verify API key is set
if openai_integration.is_api_key_set():
    print("✅ API key successfully set!")
else:
    print("❌ API key not set. Please try again or set the OPENAI_API_KEY environment variable.")

✅ API key successfully set!


## 3. Load Your CV and Cover Letter

The system will use your existing CV and cover letter as templates. Let's load them and see what they contain.

In [12]:
# List available documents in the data directory
available_documents = [f for f in os.listdir(DATA_DIR) if f.endswith('.docx')]
print("Available documents:")
for i, doc in enumerate(available_documents):
    print(f"{i+1}. {doc}")

# Default documents (you can change these if needed)
cv_filename = "Imon Hosen - Resume_Template_ATS.docx"
cover_letter_filename = "Cover Letter_Imon .docx"

# Construct full paths
cv_path = os.path.join(DATA_DIR, cv_filename)
cover_letter_path = os.path.join(DATA_DIR, cover_letter_filename)

# Load documents
cv_parser = CVParser(cv_path)
cover_letter_parser = CoverLetterParser(cover_letter_path)

# Display some information about the loaded documents
print("\nCV Information:")
personal_info = cv_parser.get_personal_info()
print(f"Name: {personal_info.get('name', 'Not found')}")
print(f"Email: {personal_info.get('email', 'Not found')}")
print(f"Phone: {personal_info.get('phone', 'Not found')}")

print("\nCV Sections:")
for section in cv_parser.get_all_sections().keys():
    print(f"- {section}")

print("\nCover Letter Information:")
print(f"Header: {cover_letter_parser.get_header() or 'Not found'}")
print(f"Greeting: {cover_letter_parser.get_greeting() or 'Not found'}")
print(f"Body length: {len(cover_letter_parser.get_body())} characters")

# Initialize document updater
document_updater = DocumentUpdater(cv_path, cover_letter_path, openai_integration)

Available documents:
1. Cover Letter_Imon .docx
2. Imon Hosen - Resume_Template_ATS.docx

CV Information:
Name: Imon Hosen
Email: imonhosen99@gmail.com
Phone: +491606355807  

CV Sections:
- HEADER
- PROFILE
- EDUCATION
- SKILLS
- CERTIFICATIONS
- EXPERIENCE
- PROJECTS

Cover Letter Information:
Header: Imon Hosen
Greeting: Not found
Body length: 0 characters


## 4. Enter Job Description

Now, paste the job description you want to tailor your CV and cover letter for:

In [18]:
# Input area for job description
job_description = """



"""

# Display the job description for confirmation
print("Job Description Preview (first 300 characters):")
print(job_description[:300] + "..." if len(job_description) > 300 else job_description)
print(f"\nTotal length: {len(job_description)} characters")

Job Description Preview (first 300 characters):



Working Student R&D - Material Master Data (all genders) 
Hamburg, Hamburg, Germany · 1 week ago · 40 people clicked apply
Hybrid  Full-time  Internship
Skills: Communication, English, +8 more
Curious where you stand? See how you compare to 40 others who clicked apply. Try Premium for €0

Apply

...

Total length: 2457 characters


## 5. Analyze Job Description

Now, let's analyze the job description to identify key requirements and skills that should be emphasized in your CV and cover letter.

In [19]:
# Analyze job description
print("Analyzing job description... (this may take a moment)")
analysis_result = document_updater.analyze_job_description(job_description)

# Display analysis results
if "error" in analysis_result:
    print(f"Error: {analysis_result['error']}")
elif "raw_response" in analysis_result:
    try:
        # Try to parse as JSON
        suggestions = json.loads(analysis_result["raw_response"])
        
        print("\n📋 Analysis Results:")
        print("\n🔹 Suggested Profile Summary:")
        print(suggestions.get("profile_summary", "No suggestions"))
        
        print("\n🔹 Key Skills to Emphasize:")
        for skill in suggestions.get("skills", []):
            print(f"- {skill}")
        
        print("\n🔹 Experience Highlights:")
        for highlight in suggestions.get("experience_highlights", []):
            print(f"- {highlight}")
        
        print("\n🔹 Keywords to Emphasize:")
        for keyword in suggestions.get("keywords_to_emphasize", []):
            print(f"- {keyword}")
    except json.JSONDecodeError:
        # If not valid JSON, display raw text
        print("\n📋 Analysis Results (raw):")
        print(analysis_result["raw_response"])
else:
    print("No analysis results returned.")

Analyzing job description... (this may take a moment)
No analysis results returned.


## 6. Generate Tailored Documents

Now, let's generate tailored versions of your CV and cover letter based on the job description analysis.

In [20]:
# Generate a timestamp for the output files
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
job_title = "Job_Application"  # You can customize this

# Define output paths
tailored_cv_path = os.path.join(OUTPUT_DIR, f"{job_title}_CV_{timestamp}.docx")
tailored_cover_letter_path = os.path.join(OUTPUT_DIR, f"{job_title}_CoverLetter_{timestamp}.docx")

# Generate tailored CV
print("Generating tailored CV...")
cv_result = document_updater.update_cv(job_description, tailored_cv_path)
print(f"✅ Tailored CV saved to: {os.path.basename(cv_result)}")

# Generate tailored cover letter
print("\nGenerating tailored cover letter...")
cover_letter_result = document_updater.update_cover_letter(job_description, tailored_cover_letter_path)
print(f"✅ Tailored cover letter saved to: {os.path.basename(cover_letter_result)}")

# Display success message with file paths
display(HTML(f"""
<div style="background-color: #e6f7e6; padding: 15px; border-radius: 5px; margin-top: 20px;">
    <h3 style="color: #2e8b57;">✅ Documents Successfully Generated!</h3>
    <p>Your tailored documents have been saved to the following locations:</p>
    <ul>
        <li><strong>CV:</strong> {tailored_cv_path}</li>
        <li><strong>Cover Letter:</strong> {tailored_cover_letter_path}</li>
    </ul>
    <p>You can now review and further customize these documents before submitting your application.</p>
</div>
"""))

Generating tailored CV...
✅ Tailored CV saved to: Job_Application_CV_20250417_003015.docx

Generating tailored cover letter...
✅ Tailored cover letter saved to: Job_Application_CoverLetter_20250417_003015.docx


## 7. (next feature)Customization Options

You can customize various aspects of the tailoring process by modifying the parameters below:

In [21]:
# OpenAI model options
def change_openai_model(model_name="gpt-3.5-turbo"):
    """Change the OpenAI model used for analysis and generation.
    
    Args:
        model_name: Name of the OpenAI model to use (e.g., "gpt-3.5-turbo", "gpt-4")
    """
    # This would require modifying the OpenAIIntegration class to accept model name as a parameter
    print(f"Model changed to: {model_name}")
    print("Note: This functionality requires updating the OpenAIIntegration class.")

# Example usage:
# change_openai_model("gpt-4")

 ## 8. (next feature)Process Multiple Job Applications

If you want to apply for multiple positions, you can use the function below to process multiple job descriptions in batch:

In [22]:
def process_multiple_jobs(job_descriptions, job_titles):
    """Process multiple job applications at once.
    
    Args:
        job_descriptions: List of job description texts
        job_titles: List of job titles (used for file naming)
    
    Returns:
        Dictionary mapping job titles to output file paths
    """
    results = {}
    
    for i, (description, title) in enumerate(zip(job_descriptions, job_titles)):
        print(f"Processing job {i+1}/{len(job_descriptions)}: {title}")
        
        # Generate timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Define output paths
        cv_path = os.path.join(OUTPUT_DIR, f"{title.replace(' ', '_')}_CV_{timestamp}.docx")
        cover_letter_path = os.path.join(OUTPUT_DIR, f"{title.replace(' ', '_')}_CoverLetter_{timestamp}.docx")
        
        # Generate tailored documents
        cv_result = document_updater.update_cv(description, cv_path)
        cover_letter_result = document_updater.update_cover_letter(description, cover_letter_path)
        
        results[title] = {
            "cv": cv_result,
            "cover_letter": cover_letter_result
        }
        
        print(f"✅ Completed: {title}\n")
    
    return results

# Example usage:
'''
job_descriptions = [
    "First job description...",
    "Second job description..."
]

job_titles = [
    "Data_Analyst",
    "Software_Engineer"
]

results = process_multiple_jobs(job_descriptions, job_titles)
'''

'\njob_descriptions = [\n    "First job description...",\n    "Second job description..."\n]\n\njob_titles = [\n    "Data_Analyst",\n    "Software_Engineer"\n]\n\nresults = process_multiple_jobs(job_descriptions, job_titles)\n'